<a href="https://colab.research.google.com/github/dcsgod/Distillbert2-finetunned/blob/main/moviemodelFTLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 \
  --index-url https://download.pytorch.org/whl/cu121

!pip install transformers==4.38.2 datasets accelerate sentencepiece


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.0 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


In [ ]:
from datasets import load_dataset, concatenate_datasets

# Movie QA (merge all versions)
movie_qa = load_dataset("HiTruong/movie_QA")

qa_ds = concatenate_datasets([
    movie_qa["train_v1"],
    movie_qa["train_v2"],
    movie_qa["train_v3"]
]).shuffle(seed=42)

# Movie metadata
meta_ds = load_dataset("Pablinho/movies-dataset", split="train")

print(len(qa_ds), len(meta_ds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


60000 9837


In [ ]:
def meta_to_qa(ex):
    return {
        "text": f"Question: What is the plot of the movie {ex['Title']}?\n"
                f"Answer: {ex['Overview']}\n"
    }

meta_qa = meta_ds.map(meta_to_qa, remove_columns=meta_ds.column_names)


In [ ]:
print(qa_ds.column_names)


['Question', 'Answer']


In [ ]:
def qa_to_text(ex):
    return {
        "text": f"Question: {ex['Question']}\nAnswer: {ex['Answer']}\n"
    }

qa_text = qa_ds.map(
    qa_to_text,
    remove_columns=qa_ds.column_names
)


In [ ]:
print(qa_text[0]["text"])


Question: What genre does the movie Tammy belong to?
Answer: The film Tammy falls under the following genres: Comedy.



In [ ]:
from datasets import concatenate_datasets

train_ds = concatenate_datasets([qa_text, meta_qa]).shuffle(seed=42)


In [ ]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

0it [00:00, ?it/s]

2.2.2+cu121
4.38.2


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "distilbert/distilbert-base-uncased-distilled-squad"
)

# Add PAD token properly
tokenizer.add_special_tokens({"pad_token": "[PAD]"})


0

In [ ]:
import torch
print(torch.cuda.is_available(), torch.cuda.get_device_name(0))


True Tesla T4


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

training_args = TrainingArguments(
    output_dir="./distilgpt2-movieqa",

    # GPU-friendly batch sizes
    per_device_train_batch_size=8,     # T4 can handle this
    gradient_accumulation_steps=2,      # effective batch = 16

    num_train_epochs=3,
    learning_rate=5e-5,

    logging_steps=100,
    save_strategy="epoch",

    # 🔥 GPU settings
    fp16=True,                          # mixed precision
    bf16=False,                         # T4 does NOT support bf16
    no_cuda=False,                      # ENABLE CUDA

    report_to="none"                    # avoid wandb popups
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
)

trainer.train()


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="./distilgpt2-movieqa/checkpoint-13095",
    tokenizer=tokenizer,
    device=0   # GPU (use -1 for CPU)
)

pipe(
    "Question: Who directed Inception?\nAnswer:",
    max_length=80,
    do_sample=True,
    temperature=0.7
)


In [ ]:
FINAL_DIR = "./distilgpt2-movieqa-final"

trainer.save_model(FINAL_DIR)
tokenizer.save_pretrained(FINAL_DIR)


In [ ]:
!ls distilgpt2-movieqa-final


In [ ]:
!zip -r distilgpt2-movieqa-final.zip distilgpt2-movieqa-final


In [ ]:
from huggingface_hub import login
login()


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(FINAL_DIR)
tokenizer = AutoTokenizer.from_pretrained(FINAL_DIR)

model.push_to_hub("iravikr/distilgpt2-movieqa")
tokenizer.push_to_hub("iravikr/distilgpt2-movieqa")
